<a href="https://colab.research.google.com/github/LEFT-BEE/KMU-Algorithm/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%EA%B3%BC%ED%95%99/naviebayes_pratice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 전처리

In [1]:
import requests
import zipfile as zipfile
import matplotlib.pyplot as plt
import pandas as pd
import re
import random
from collections import Counter

In [6]:
#데이터 받아오기 , zip풀기
url = "https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip"
r = requests.get(url)
open("spam.zip", "wb").write(r.content)
zipfile.ZipFile("spam.zip").extractall("./")
pd.read_csv("SMSSpamCollection.txt" , sep = "\t" , names = ["cls" , "txt"])

In [8]:
data = [line.strip().split("\t") for line in open("SMSSpamCollection.txt", "r")]
for d in data: #d[1] is data
 d[1] = set(re.findall("[a-z0-1_]+", d[1].lower()))

random.shuffle(data)
train_len = int(len(data) * 0.8)
test_len = len(data) - train_len
train = data[:train_len]
test = data[train_len:]

## 필요한 데이터 구하기

In [9]:
#train데이터에서 naive nayes 계산을 위한 값들을 계한해놓기
#priror prob , p(spam) , p(ham)
#라플라스 스무딩 -> 관측한 메일에서 한번도 등장하지 않은 단어가 있으면, 예측할 때 결과가 이상해지는 문제 해결
#하나의 메일에 모든 단어가 하나씩 포함되어있음
alpha = 0.1

n_total = len(train)
n_spam = sum(1 for cls,bow in train if cls =="spam")
n_ham = sum(1 for cls,bow in train if cls =="ham")
#사전확률 구하기
prior_spam = (n_spam+alpha) / (n_total+ 2*alpha)
prior_ham = (n_ham+alpha) / (n_total+ 2*alpha)

In [19]:
# 단어의 가능도 P(w|spam) , p(w|ham)
spam_words= {}
ham_words = {}
for cls , bow in train:
  for word in bow:
    if cls == "spam":
      if word not in spam_words:
        spam_words[word] = 0
      spam_words[word] +=1
    else:
      if word not in ham_words:
        ham_words[word] = 0
      ham_words[word] += 1

print(f" buy 단어의 등장 횟수 : spam - {spam_words['buy']} , ham - {ham_words['buy']}")

  단어의 등장 횟수 : spam - 4 , ham - 38


In [20]:
#counter를 이용하여 각각의 단어가 얼마나 들어있는지 확인 
spam_words = Counter(word for cls , bow in train
  for word in bow
  if cls =='spam')

ham_words = Counter(word for cls , bow in train
  for word in bow
  if cls =='ham')

## 예측하기

In [21]:
def predict(bow):
  #p(W|spam) = w를 포함한 spam 수 + alpha / 전체 spam수 +alpha 
  #spam_score = P(spam) * P(w1|spam) * p(w2|spam) ...
  
  spam_score = prior_spam
  ham_score  = prior_ham
  for word in bow:
    spam_score *=(spam_words[word] + alpha) / (n_spam+alpha)
    ham_score *= (ham_words[word] +alpha) / (n_ham + alpha)

  if spam_score < ham_score:
    return "ham"
  else:
    return "spam"

In [23]:
#confusion matrix
tp , tn , fp ,fn = 0,0,0,0
for ans ,bow in test:
  pred = predict(bow)
  if pred == 'spam':
    if ans == 'spam':
      tp +=1
    else:
      fp +=1
  else:
    if ans == 'ham':
      tn +=1
    else:
      fn +=1
# 전체 테스트 수 중 맞힌 수 
accuracy = (tp + tn) / (tp+tn+fp+fn)
#실제스팸 / 스팸예측
precision =  tp / (tp + fp)
# 내가 찾아낸 스팸수 / 실제 스팸 수 
recall = tp /(tp + fn)


f1 = 2 *precision * recall / (precision + recall)
print(f"F1 soore is {round(f1,3)}%")

F1 soore is 0.837%
